## Data Extraction

In [1]:
import cloudscraper
from bs4 import BeautifulSoup
import datetime as dt
import re
import time
import calendar
import os
import numpy as np
from dateutil.parser import *
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
from pytz import timezone

In [3]:
#returns the pkl file name of certain pair to standardize
def get_his_data_filename(pair, granularity,time_from = None, time_to = None):
    
    if time_from:
        
        return f"./datasets/{pair}_{granularity}_{time_from}_{time_to}.pkl"
        
    else:
        return f"./datasets/{pair}_{granularity}.pkl"

#function to parse time from str to dt
    
def parser(row):
    d= parse(row)
    return d

In [4]:
def timeDateAdjust(event_time_hour, event_time_minutes, am_or_pm, hours_to_adjust, year, month, day):
    
    # function to change to 24h format and adjust to local timezone
    
    # Hours_to_adjust input is used to adjust for timzone differences as the forex factory calendar is in EST
    
    d = dt.date(year, month, day)
    
    if(am_or_pm == "am"):
        adjusted_hour = int(event_time_hour) + hours_to_adjust
    
    else:
    # If pm then add 12 hours to adjust to 24 hours format
        adjusted_hour = int(event_time_hour) + 12 + hours_to_adjust
    
    if (adjusted_hour < 24):
    # If adjusted_hour < 24 hours no need to update the date
    # if it is over 24 then this means that it is the next day and the date needs to be updated.
    
        adjusted_time = str(adjusted_hour) + event_time_minutes # Returns string representation of the 24h time in HH:MM
        d_of_week = calendar.day_abbr[d.weekday()] # use the calendar API to return Mon-Sun in abbreviated format as a string
        d= d.strftime("%Y.%m.%d") # Returns the date as a string in the format YYYY:MM:DD
        
        # returns date, time and day of week
        return (d, adjusted_time, d_of_week)
    
    else:
        
        adjusted_hour = adjusted_hour - 24 
        # If it is PM, Minus 24h as it is now the next day and differenced time will be am time of the next day
        
        adjusted_time = str(adjusted_hour) + event_time_minutes 
        # Returns string representation of the 24h time in HHMM
        
        d = d + dt.timedelta(days=1) 
        # Adds one day on the original date of the event as it is now new day of local time
        
        d_of_week = calendar.day_abbr[d.weekday()] # use the calendar API to return Mon-Sun in abbreviated format as a string
        d= d.strftime("%Y.%m.%d") # Returns the date as a string in the format YYYY:MM:DD
        
        # returns date, time and day of week
        return (d, adjusted_time, d_of_week)
        
def strToIntMonth(month):

    # Function to convert Str Month into an Int

    if (month == 'Jan'):
        return 1
    elif (month == "Feb"):
        return 2
    elif (month == "Mar"):
        return 3
    elif (month == "Apr"):
        return 4
    elif (month == "May"):
        return 5
    elif (month == "Jun"):
        return 6
    elif (month == "Jul"):
        return 7
    elif (month == "Aug"):
        return 8
    elif (month == "Sep"):
        return 9
    elif (month == "Oct"):
        return 10
    elif (month == "Nov"):
        return 11
    elif (month == "Dec"):
        return 12
    else:
        return None

In [5]:
def getEventsCalendar(start_date, end_date, file_path):
    
        # Need cloudscraper to bypass cloudflare
        scraper = cloudscraper.create_scraper(allow_brotli=False, disableCloudflareV1=True)

        # Gets One Day at a time

        url = 'https://www.forexfactory.com/' + start_date

        # query the website and return the html to the variable ‘page’
        page = scraper.get(url).text

        ### print(page)
        # parse the html using beautiful soup and store in variable `soup`

        soup = BeautifulSoup(page, 'html.parser')

        # Find the table containing all the data
        table = soup.find('table', class_ = 'calendar__table')

        # Get Date of Event
        #print(table.find_next('tr', class_ = 'calendar__row--new-day'))
        date_of_events = table.find_next('tr', class_ = 'calendar__row--new-day').find_next('span', class_ = 'date')

        # Regualr Expression to find the 'day of week', 'month' and the 'day'
        matchObj = re.search('([a-zA-Z]{3})([a-zA-Z]{3}) ([0-9]{1,2})', date_of_events.text)

        # Assigning the 'day of week', 'month' and 'day'

        day_of_week = matchObj.group(1)
        month = matchObj.group(2)
        month = strToIntMonth(month) # Convert from Str to Int
        month = int(format(month, '02')) # Places 0's in front of the month if it is single digit day, for months Jan - Sep

        day = matchObj.group(3)
        day = int(format(int(day), "02")) # Places 0's in front of the day if it is single digit day, for days 1-9 of the month

        year = int(start_date[-4:])

        date_tentative = np.NaN

        # Event Times 
        event_times = table.find_all('td', class_ = 'calendar__time')

        if(day_of_week != 'Sat' and day_of_week != 'Sun'):
            for news in event_times:

                # currency which will be effected by the event
                curr = news.find_next_sibling('td', class_ = 'currency').text.strip()

                # impact of event (high,medium,low)
                impact = news.find_next_sibling('td', class_ = 'impact').find_next('span')['class']
                impact = impact[0]

                #event name

                event = news.find_next_sibling('td', class_ = 'event').find_next('span').text.strip()

                #previous report figures if it is a continuous on-going report e.g. nfp

                previous = news.find_next_sibling('td', class_ = 'previous').text

                #forecasted figures if it is a continuous on-going report e.g. nfp

                forecast = news.find_next_sibling('td', class_ = 'forecast').text

                # actual figures if event has happened

                actual = news.find_next_sibling('td', class_ = 'actual').text

                # get event time
                event_time = news.text.strip()

                try:
                    matchObj = re.search('([0-9]+)(:[0-9]{2})([a|p]m)', event_time) # Regex to match time in the format HH:MMam/pm

                    if(matchObj != None):

                    # if we get a time, then we store the hour, minutes and am/pm

                        event_time_hour = matchObj.group(1) # Matches the first group in the regex which is the hour in HH format
                        event_time_minutes = matchObj.group(2) # Matches the second group in the regex which is the minutes in :MM format 
                        am_or_pm = matchObj.group(3) # Matches the third group in the regex which is either 'am' or 'pm'

                    # for events that are all day events, we cannot trade on them and put the time as nan

                    elif(re.search('([a-zA-Z]+)', event_time)):
                        with open(file_path, 'a') as file:
                            file.write('{}, {}, {}, {}, {}, {}, {}, {}, {}\n'.format(day_of_week, date_tentative, event_time, curr, impact, event, previous, forecast, actual))
                        continue

                    else:

                        # else no time and use previous events time and write to file as ff will not give multiple 
                        # \n event times if they are of the same timing

                        with open(file_path, 'a') as file:
                            file.write('{}, {}, {}, {}, {}, {}, {}, {}, {}\n'.format(day_of_week, event_date_time, event_time_holder, curr, impact, event, previous, forecast, actual))
                        continue
                        
                        #parser(event_date_time)

                    # adjust the datetime to 24h and local time
                    # Returns a tuple with 3 elements consisting of 'event date YYYY:MM:DD', 'event time HH:MM', 
                    # \n 'day of week Mon-Fri'

                    adjusted_date_time = timeDateAdjust(event_time_hour, event_time_minutes, am_or_pm, 12, year, month, day)

                    event_date = adjusted_date_time[0]
                    event_time = adjusted_date_time[1]
                    day_of_week = adjusted_date_time[2]

                    if event_time != '' and event_time != 'All Day': # If the event time is not empy and not 'All day' then we have found a time 
                        event_time_holder = str(adjusted_date_time[1]) # Set the event_time_holder to this event_time so any other same time events can use this to record
                                                                    #\n the same time as forex factory only provides a time for the first event
                        event_date_time = '{} {}'.format(event_date, event_time_holder)
                    else:
                        event_time_holder = event_time_holder # event_time_holder remains the same and should have the value of the first event which was assigned a time
                        event_date_time = '{} {}'.format(event_date, event_time_holder) 

                except Exception as e:
                    print("There was an error: " + e)

                ### print(file_path)
                print('{}, {}, {}, {}, {}, {}, {}, {}, {}\n'.format(day_of_week, event_date_time, event_time_holder, curr, impact, event, previous, forecast, actual))
                with open(file_path, 'a') as file:
                    file.write('{}, {}, {}, {}, {}, {}, {}, {}, {}\n'.format(day_of_week, event_date_time, event_time_holder, curr, impact, event, previous, forecast, actual))

        if start_date == end_date:
            print('Successfully retrieved all data')
            return True

        else:
            scrape_next_day = soup.find('div', class_='head').find_next('a', class_='calendar__pagination--next')['href']
            getEventsCalendar(scrape_next_day, end_date, file_path)

In [155]:
#command to scrape
#take note that if u query too many times, cloudfare might detect and block u. u may have to use vpn then.

# event_time_holder = '' # Holds event time of previous news event if it does not have one

# abs_path = os.path.abspath('a')
# cwd = os.path.dirname(abs_path)
# parent_dir = os.path.dirname(cwd)  
# file_path = parent_dir + "/capstone/ffc_news_events_2.csv"
# ### file_path = '/Users/User_1/Desktop' + "/ffc_news_events.csv"

# os.makedirs(os.path.dirname(file_path), exist_ok=True) #create file if file does not exist

# with open(file_path, 'w') as file:
#     file.write("") # Needs to write an empty line so that file is opened and getEventsCalendar can append to the file

# est_tz = timezone('EST') #query in est

# est_date_now = dt.datetime.now(est_tz)

# days_into_future = 8

# est_date_future = dt.datetime.now(est_tz)+dt.timedelta(days_into_future)

# date_start = str(calendar.month_abbr[est_date_now.month]) + str(est_date_now.day)+ '.' + str(est_date_now.year)
# date_end = str(calendar.month_abbr[est_date_future.month]) + str(est_date_future.day)+ '.' + str(est_date_future.year)

# getEventsCalendar(f"calendar?day={date_start}".lower(),f"calendar?day={date_end}".lower(), file_path)

Mon, 2022.05.09 14:45, 14:45, EUR, low, French Trade Balance, -10.4B, -11.2B, -12.4B

Mon, 2022.05.09 16:30, 16:30, EUR, low, Sentix Investor Confidence, -18.0, -21.7, -22.6

Mon, 2022.05.09 20:30, 20:30, CAD, low, Building Permits m/m, 21.0%, 3.4%, -9.3%

Mon, 2022.05.09 21:00, 21:00, GBP, medium, MPC Member Saunders Speaks, , , 

Mon, 2022.05.09 22:00, 22:00, USD, low, Final Wholesale Inventories m/m, 2.3%, 2.3%, 

Tue, 2022.05.10 7:01, 7:01, GBP, low, BRC Retail Sales Monitor y/y, -0.4%, 3.5%, 

Tue, 2022.05.10 7:30, 7:30, JPY, low, Household Spending y/y, 1.1%, -3.2%, 

Tue, 2022.05.10 9:30, 9:30, AUD, low, NAB Business Confidence, 16, , 

Tue, 2022.05.10 11:35, 11:35, JPY, low, 10-y Bond Auction, 0.20|3.6, , 

Tue, 2022.05.10 16:00, 16:00, EUR, low, Italian Industrial Production m/m, 4.0%, -1.4%, 

Tue, 2022.05.10 17:00, 17:00, EUR, low, ZEW Economic Sentiment, -43.0, -42.0, 

Tue, 2022.05.10 18:00, 18:00, USD, low, NFIB Small Business Index, 93.2, 92.9, 

Tue, 2022.05.10 19:40, 1

In [157]:
###pd.read_csv('./datasets/ffc_news_events_2.csv')

In [7]:
# extracting events from saved file

event_df = pd.read_csv('./datasets/ffc_news_events_2.csv')

event_df.columns = ['day_of_week', 'event_date_time', 'event_time_holder', 'curr', 'impact', 'event', 'previous', 'forecast', 'actual']

event_df = event_df.select_dtypes(['object']).apply(lambda x: x.str.strip())

# exclude all day events which we have stored as nan

event_df = event_df[event_df.event_date_time != 'nan']

event_df.reset_index(inplace=True,drop=True)

event_df['event_date_time']= event_df['event_date_time'].apply(parse)

display(event_df)

### event_df = event_df[( (event_df['curr']=='USD')| (event_df['curr']=='EUR')) & (event_df['impact']!= 'low')]

# we exclude low impact events and only take in medium and high impact events
event_df = event_df[((event_df['curr']=='USD')) & (event_df['impact']!= 'low')]

event_df.reset_index(inplace=True,drop=True)

### event_df.to_pickle('ff_event_df_010115_033122.pkl')

,day_of_week,event_date_time,event_time_holder,curr,impact,event,previous,forecast,actual
0,Wed,2022-05-11 14:00:00,14:00,EUR,low,German Final CPI m/m,0.8%,0.8%,0.8%
1,Wed,2022-05-11 15:15:00,15:15,EUR,low,German Buba President Nagel Speaks,,,
2,Wed,2022-05-11 16:00:00,16:00,EUR,medium,ECB President Lagarde Speaks,,,
3,Wed,2022-05-11 17:32:00,17:32,EUR,low,German 10-y Bond Auction,0.93|1.6,,0.96|1.4
4,Wed,2022-05-11 20:30:00,20:30,USD,high,CPI m/m,1.2%,0.2%,0.3%
5,Wed,2022-05-11 20:30:00,20:30,USD,high,Core CPI m/m,0.3%,0.4%,0.6%
6,Wed,2022-05-11 22:30:00,22:30,USD,low,Crude Oil Inventories,1.3M,-1.0M,8.5M
7,Thu,2022-05-12 01:01:00,1:01,USD,low,10-y Bond Auction,2.72|2.4,,2.94|2.5
8,Thu,2022-05-12 01:15:00,1:15,USD,low,President Biden Speaks,,,
9,Thu,2022-05-12 02:00:00,2:00,USD,low,Federal Budget Balance,-192.7B,218.5B,308.2B


In [45]:
API_KEY = 'x'
ACCOUNT_ID = 'x'

OANDA_URL = 'https://api-fxpractice.oanda.com/v3'

SECURE_HEADER = {
    'Authorization': f'Bearer {API_KEY}',
    'Content-Type': 'application/json'
}

In [47]:
class OandaAPI():
    #any oandaapi object that queries object.session will be calling requests.session
    def __init__(self):
        self.session = requests.Session()    
        
    #get currency candles data from server, default count is none, granularity is 1 minute
    def fetch_candles(self, pair_name, count=None, granularity="M1", date_from=None, date_to=None, as_df=False):
        url = f"{OANDA_URL}/instruments/{pair_name}/candles"

        params = dict(
            granularity = granularity,
            price = "MBA"
        )
        
        if date_from is not None and date_to is not None:
            params['to'] = int(date_to.timestamp())
            params['from'] = int(date_from.timestamp())
        elif count is not None:
            params['count'] = count
        else:
            params['count'] = 300
            
        try:
            response = self.session.get(url, params=params, headers=SECURE_HEADER)
        
        # sometimes may have error getting candles, so we retry again with sleep
        
        except:
            print('GET error, sleep')
            time.sleep(3)
            response = self.session.get(url, params=params, headers=SECURE_HEADER)
            print (response.status_code)
            
        if response.status_code != 200:
            return response.status_code, None
        
        # convert candles to df
        if as_df == True:
            try:
                json_data = response.json()['candles']
                return response.status_code, OandaAPI.candles_to_df(json_data)
            except:
                print('sleep')
                time.sleep(3)
                response = self.session.get(url, params=params, headers=SECURE_HEADER)
                json_data = response.json()['candles']
                print (response.status_code)
                return response.status_code, OandaAPI.candles_to_df(json_data)
        else:        
            return response.status_code, response.json()

    @classmethod
    # function to store candles as df
    def candles_to_df(cls, json_data):
        prices = ['mid', 'bid', 'ask']
        ohlc = ['o', 'h', 'l', 'c']

        our_data = []
        for candle in json_data:
            if candle['complete'] == False:
                continue
            new_dict = {}
            new_dict['time'] = candle['time']
            new_dict['volume'] = candle['volume']
            for price in prices:
                for oh in ohlc:
                    new_dict[f"{price}_{oh}"] = float(candle[price][oh])
            our_data.append(new_dict)
        df = pd.DataFrame.from_dict(our_data)
        return df

In [48]:
# function to save candles to file

INCREMENTS = {'M1':1, 'M5':5, 'H1':60,
             'D':1440}

pair = 'XAU_USD'
granularity = 'M1'

def create_file(pair,granularity,api,date_from,end_date,time_from = None,time_to = None):
    
    candle_count = 2000
    
    time_step = INCREMENTS[granularity] * candle_count
    
    candle_dfs = []
    
    while date_from < end_date:
        date_to = date_from + dt.timedelta(minutes = time_step)
    
        if date_to > end_date:
            date_to = end_date
        
        code, df = api.fetch_candles(pair, granularity=granularity, date_from = date_from, date_to = date_to, as_df = True)
        
        if df is not None and df.empty == False:
            candle_dfs.append(df)
            
        elif code != 200:
            print ('ERROR', pair, granularity, date_from,date_to)
                
            break
            
        date_from = date_to # increase timestep to get next batch of candles
        
    final_df = pd.concat(candle_dfs)
    
    final_df.drop_duplicates(subset = 'time', inplace=True)
    
    final_df.sort_values(by = 'time', inplace=True)
    
    # if we want to include the time in the file name
    
    if time_to:
        
        final_df.to_pickle(f'{get_his_data_filename(pair,granularity,time_from,time_to)}')
  
    else:
        final_df.to_pickle(f'{get_his_data_filename(pair,granularity)}')
    
    print (f'{pair} {granularity} {final_df.iloc[0].time} {final_df.iloc[-1].time} {dt.datetime.now().hour}:{dt.datetime.now().minute}')

In [ ]:
# extracting gold candles for day for sarimax
create_file('XAU_USD',granularity= 'D',api = OandaAPI(),date_from = parser('2015-01-01'),
            end_date = parser('2022-03-31'),)

# extracting silver candles for day for sarimax
create_file('XAG_USD',granularity= 'D',api = OandaAPI(),date_from = event_df['event_date_time'].iloc[0],
            end_date = parser('2022-03-31'),)

In [ ]:
ts_df = pd.read_pickle(get_his_data_filename('XAU_USD','D'))

ts_df.to_pickle('./datasets/XAU_USD_D_010115_300322.pkl')

ts_df_sil = pd.read_pickle(get_his_data_filename('XAG_USD','D'))

ts_df_sil.to_pickle('./datasets/XAG_USD_D_010115_300322.pkl')

In [ ]:
#creating extra candles outside test for validation/backtesting of sarimax

create_file('XAU_USD',granularity= 'D',api = OandaAPI(),date_from = parser('2015-01-01'),
            end_date = parser('2022-04-18')+dt.timedelta(30),time_from = '2015',time_to = '180422')

In [ ]:
# extracting gold minute candles
# our event is until march so we extract additional candles till may for backtesting
create_file('XAU_USD',granularity= 'M1',api = OandaAPI(),date_from = parser('2015-01-01'),
            end_date = parser('2022-04-28'),)

In [11]:
xau_usd_M1_2015_042822 = pd.read_pickle(get_his_data_filename('XAU_USD','M1'))

xau_usd_M1_2015_042822.to_pickle('./datasets/xau_usd_M1_2015_042822.pkl')